Subiectul 1.

a.

In [18]:
import pandas as pd
import pymc as pm
import numpy as np
import arviz as az
data = pd.read_csv("date_alegeri_turul2.csv")

x1 = data["Varsta"].values
x2 = data["Sex"].values
x3 = data["Educatie"].values
x4 = data["Venit"].values
y = data["Vot"].values

x1 = (x1 - np.mean(x1)) / np.std(x1)
x4 = (x4 - np.mean(x4)) / np.std(x4)

with pm.Model() as model:
    beta0 = pm.Normal("beta0", mu=0, sigma=10)
    beta1 = pm.Normal("beta1", mu=0, sigma=10)
    beta2 = pm.Normal("beta2", mu=0, sigma=10)
    beta3 = pm.Normal("beta3", mu=0, sigma=10)
    beta4 = pm.Normal("beta4", mu=0, sigma=10)
    
    p = pm.math.sigmoid(beta0 + beta1 * x1 + beta2 * x2 + beta3 * x3 + beta4 * x4)
    
    y_obs = pm.Bernoulli("y_obs", p=p, observed=y)
    
    trace = pm.sample(2000, tune=1000, target_accept=0.95)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1, beta2, beta3, beta4]


Output()

Sampling 4 chains for 1_000 tune and 2_000 draw iterations (4_000 + 8_000 draws total) took 25 seconds.


b)

In [19]:
beta_means = az.summary(trace, var_names=["beta1", "beta2", "beta3", "beta4"])["mean"]

top_2 = beta_means.abs().sort_values(ascending=False).index[:2]
print("Cele mai influente variabile:", top_2)

Cele mai influente variabile: Index(['beta4', 'beta2'], dtype='object')


Din rezultatul de mai sus rezulta faptul ca cea mai influente 2 variabile sunt beta2 (Sex) si beta4 (Venit) deoarece acestea au HDI-ul cel mai ridicat

c)

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import pymc as pm
import arviz as az

x1 = data['Sex'] 
x2 = data['Venit']
y = data['Vot']

with pm.Model() as logistic_model:
    beta0 = pm.Normal("beta0", mu=0, sigma=10)
    beta1 = pm.Normal("beta1", mu=0, sigma=10)
    beta2 = pm.Normal("beta2", mu=0, sigma=10)
    
    p = pm.math.sigmoid(beta0 + beta1 * x1 + beta2 * x2)
    y_obs = pm.Bernoulli("y_obs", p=p, observed=y)
    
    trace = pm.sample(1000, tune=1000, return_inferencedata=True)

beta0_mean = trace.posterior['beta0'].mean().item()
beta1_mean = trace.posterior['beta1'].mean().item()
beta2_mean = trace.posterior['beta2'].mean().item()



Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta0, beta1, beta2]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 23 seconds.


Subiectul 2

In [ ]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, davies_bouldin_score

data = pd.read_csv("iris.csv")
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
results = {}

for feature in features:
    X = data[[feature]].values
    gmm = GaussianMixture(n_components=3, random_state=42)
    clusters = gmm.fit_predict(X)

    silhouette = silhouette_score(X, clusters)
    davies_bouldin = davies_bouldin_score(X, clusters)

    results[feature] = {
        "silhouette_score": silhouette,
        "davies_bouldin_score": davies_bouldin
    }

best_feature = max(results, key=lambda f: results[f]['silhouette_score'])
print("Cea mai buna caracteristica pentru separare:", best_feature)
print("Rezultatele complete:", results)


Cea mai buna caracteristica pentru separare: petal_width
Rezultatele complete: {'sepal_length': {'silhouette_score': 0.5608928970606418, 'davies_bouldin_score': 0.5282131804268166}, 'sepal_width': {'silhouette_score': 0.5641985631061969, 'davies_bouldin_score': 0.5190101059463758}, 'petal_length': {'silhouette_score': 0.6781767245923337, 'davies_bouldin_score': 0.4487565687151886}, 'petal_width': {'silhouette_score': 0.7268315079591691, 'davies_bouldin_score': 0.39278522884998607}}
